[PyImageSearch](https://www.pyimagesearch.com/) has a really good list of OpenCV tutorials. 
We will be using [this](https://www.pyimagesearch.com/2015/09/14/ball-tracking-with-opencv/) tutorial form that site to figure out ball tracking. 

In [ ]:
# Import necessary Python libraries 
# OpenCV library
import cv2
# Python Imaging Library useed to load images from files, and to create new images
import PIL.Image
# BytesIO implements read and write bytes data in memory
from io import BytesIO
# Library to display stuff and clear stuff on Jupyer Notebook 
import IPython.display
from IPython.display import clear_output
# Library that makes calls to openCV a bit more convinient 
import imutils
# Useful library for array manipulation 
import numpy as np
# Time library for delay 
import time
# Library to integrate buttons and user inputs
from ipywidgets import widgets
# Import library for threading 
import datetime, threading

from collections import deque

In [ ]:
#Function that converts array data to image
def array_to_image(a, fmt='jpeg'):
    # Create binary stream object
    f = BytesIO()
    
    # Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)
    
    return IPython.display.Image(data=f.getvalue())

In [ ]:
# Function to read the frame form camera
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    #flip image for natural viewing
    #frame = cv2.flip(frame, 1)

    return frame

In [ ]:
# Display the image
d1 = IPython.display.display("Your streaming will be displayed here", display_id=1)

In [ ]:
# define the lower and upper boundaries of the "green"
# ball in the HSV color space, then initialize the
# list of tracked points
# HSV (0-180,0-255,0-255)
greenLower = (49, 0, 0)
greenUpper = (98, 204, 255)
#redLower = (7, 0, 0)
#redUpper = (98, 255, 255)

buffer = 5
pts = deque(maxlen=buffer)

In [ ]:
# Start video capture 
cam = cv2.VideoCapture(0)

In [ ]:
# keep looping
for i in range(200):
    # Grab the frame 
    img = get_frame(cam)
    
    # Change the color to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Resize the image to 200px
    img = imutils.resize(img, width=200, inter=cv2.INTER_LINEAR)
    
    # resize the frame, blur it, and convert it to the HSV
    # color space
    #img = imutils.resize(img, width=200, inter=cv2.INTER_LINEAR)
    blurred = cv2.GaussianBlur(img, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # construct a mask for the color "green", then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    #mask = cv2.inRange(hsv, redLower, redUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # find contours in the mask and initialize the current
    # (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        # only proceed if the radius meets a minimum size
        if radius > 10:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(img, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
    # update the points queue
    pts.appendleft(center)
    
    # loop over the set of tracked points
    for i in range(1, len(pts)):
        # if either of the tracked points are None, ignore
        # them
        if pts[i - 1] is None or pts[i] is None:
            continue
        # otherwise, compute the thickness of the line and
        # draw the connecting lines
        thickness = int(np.sqrt(buffer / float(i + 1)) * 2.5)
        cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), thickness)
    # show the frame to our screen
    #Call the function to convert array data to image
    frame = array_to_image(img)
    d1.update(frame)
    
# Release the camera
cam.release()


In [ ]:
# Release the camera
cam.release()